Step 1 - Import necassary modules

In [8]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import linear_model
from sklearn.cross_validation import LeaveOneOut, cross_val_predict
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

from chris_rasmus_graph_features import *
from count_lagged_complaints import *
from processing import *

Step 2 - Load data

In [9]:
offc = pd.read_csv('/Users/Mehraveh/Desktop/policy_lab/toy.officer_data.csv',sep=',')
cmpl = pd.read_csv('/Users/Mehraveh/Desktop/policy_lab/toy.complaint_data.csv', sep=',')

Step 3 - Feature extraaction/insertion

In [10]:
# Adds the number of complaints done by each officer in previous years (lags)
last_train_year = 2012
number_of_years_to_go_back = 4
r = add_complaints_by_year(last_train_year, number_of_years_to_go_back, cmpl, offc) 

# Making X vector (features)
X = r

#Dropping "FirstName", "LastName", "BirthYear","Rank"
X = X.drop(X.columns[[0,1,5,7,11]], axis=1)
# Getting years from apponted dates and normalizing
# temp = X[X.columns[0]].astype(str).str[0:4].astype(int)
# X[X.columns[0]] = (temp-np.mean(temp))/np.std(temp)
X[X.columns[0]] = X[X.columns[0]].astype(str).str[0:4].astype(int)


#Encoding categorical variables
X_cat = X[X.columns[[1,2]]]
XX=X_cat.apply(LabelEncoder().fit_transform)
X[X.columns[[1,2]]] = XX
np.where(np.isnan(X))
X_race = pd.get_dummies(X['race'])
X_race.columns = ['race0','race1','race2','race3']
X = X.drop(X.columns[[1]], axis=1)
X = X.join(X_race, how='outer')




Y = add_complaints_by_year(2015, 3, cmpl, offc)

Y=Y[[Y.columns[Y.shape[1]-1],Y.columns[Y.shape[1]-2],Y.columns[Y.shape[1]-3]]]
Y=Y.sum(axis=1)


In [ ]:
#Adding graph features

scale_days = 365
base_year = 2012
last_train_year = 2012
base_month = 12
base_day = 31
complaint_df = add_lag_to_complaints(cmpl, scale_days, base_year, base_month, base_day)

# build bipartite graph
G = build_bipartite_graph(complaint_df)

lag = 4
officer_ids = offc['officer_id'].unique()
deg_thresh = 5

# get feature dictionaries
num_nbr_complaints_dict = num_of_nbr_complaints(G, officer_ids, lag)
num_high_offender_nbrs = num_high_offender_nbrs(G, officer_ids, deg_thresh)

# 

# num_nbr_complaints_dict[]

In [38]:

A=pd.DataFrame(dict([(k,pd.Series(v)) for k,v in num_nbr_complaints_dict.items()])).transpose()
A.columns = ['cmpl0','cmpl1','cmpl2','cmpl3']

B=pd.DataFrame(dict([(k,pd.Series(v)) for k,v in num_high_offender_nbrs.items()])).transpose() 
B.columns = ['num_high_offndr']

r_A = r.merge(A, left_on = 'officer_id', right_index = 1, how = 'outer')
r_AB = r_A.merge(B, left_on = 'officer_id', right_index = 1, how = 'outer')
r_AB.fillna(0);


# merge numpy array with dataframe that I create


# # merge that 


# A = pd.DataFrame.from_dict(num_high_offender_nbrs.items())

,first.name,last.name,appointed.date,race,gender,birth.year,age,rank,primary,secondary,...,officer_id,Lag0,Lag-1,Lag-2,Lag-3,cmpl0,cmpl1,cmpl2,cmpl3,num_high_offndr
0,KARINA,AARON,2005-09-26,HISPANIC,FEMALE,1980,35.0,POLICE OFFICER,0,1,...,1139,0.0,3.0,2.0,2.0,9,10,9,8,12.0
1,AZIZ,ABDELMAJEID,2008-04-28,A_OTHER,MALE,1984,31.0,POLICE OFFICER,0,0,...,434,0.0,0.0,5.0,4.0,17,25,24,18,22.0
2,JENNIFER,ACCARDO,2004-11-29,WHITE,FEMALE,1979,37.0,POLICE OFFICER,0,0,...,2515,0.0,1.0,0.0,0.0,1,1,1,1,1.0
3,ROSEMARY,ACCARDO,1986-08-11,WHITE,FEMALE,1952,63.0,POLICE OFFICER,0,0,...,1149,0.0,0.0,0.0,1.0,0,0,1,1,0.0
4,THOMAS,ACCARDO,2001-04-30,WHITE,MALE,1967,49.0,POLICE OFFICER,0,1,...,2134,1.0,0.0,1.0,0.0,1,1,0,0,0.0
5,AARON,ACEVEDO,2009-12-16,HISPANIC,MALE,1982,33.0,POLICE OFFICER,0,0,...,2747,2.0,0.0,0.0,0.0,0,1,0,0,0.0
6,ALEJANDRO,ACEVEDO,2005-09-26,HISPANIC,MALE,1972,44.0,POLICE OFFICER,0,0,...,1460,0.0,0.0,1.0,0.0,2,4,1,1,1.0
7,BIENVENID,ACEVEDO,1999-03-08,HISPANIC,MALE,1972,44.0,POLICE OFFICER,0,0,...,1290,0.0,1.0,0.0,1.0,4,3,3,2,3.0
8,JANETE,ACEVEDO,2009-12-16,HISPANIC,FEMALE,1980,35.0,POLICE OFFICER,0,0,...,2300,0.0,2.0,0.0,0.0,15,17,21,15,15.0
9,JASON,ACEVEDO,2002-07-29,HISPANIC,MALE,1978,38.0,POLICE OFFICER,0,0,...,2262,3.0,2.0,0.0,0.0,3,8,9,3,4.0


In [35]:
r_AB.head()


,first.name,last.name,appointed.date,race,gender,birth.year,age,rank,primary,secondary,...,officer_id,Lag0,Lag-1,Lag-2,Lag-3,cmpl0,cmpl1,cmpl2,cmpl3,num_high_offndr
0,KARINA,AARON,2005-09-26,HISPANIC,FEMALE,1980,35.0,POLICE OFFICER,0,1,...,1139,0.0,3.0,2.0,2.0,9,10,9,8,12.0
1,AZIZ,ABDELMAJEID,2008-04-28,A_OTHER,MALE,1984,31.0,POLICE OFFICER,0,0,...,434,0.0,0.0,5.0,4.0,17,25,24,18,22.0
2,JENNIFER,ACCARDO,2004-11-29,WHITE,FEMALE,1979,37.0,POLICE OFFICER,0,0,...,2515,0.0,1.0,0.0,0.0,1,1,1,1,1.0
3,ROSEMARY,ACCARDO,1986-08-11,WHITE,FEMALE,1952,63.0,POLICE OFFICER,0,0,...,1149,0.0,0.0,0.0,1.0,0,0,1,1,NaN
4,THOMAS,ACCARDO,2001-04-30,WHITE,MALE,1967,49.0,POLICE OFFICER,0,1,...,2134,1.0,0.0,1.0,0.0,1,1,0,0,NaN
